# Introduction to Patent_Client

The patent_client package provides easy access to a number of USPTO and EPO patent data resources, all through a familiar ORM-like interface. If you have experience with the [Django ORM](https://docs.djangoproject.com/en/2.1/topics/db/queries/#retrieving-objects) with some extra features to accomodate the various data sources. Each data source is encapsulated in a Model object that represents various API's. The top-level models are:

- `USApplication` (USPTO Patent Examination Data System)
- `PtabTrial` (USPTO PTAB Trials API)
- `Assignment` (USPTO Assignments API)
- `Inpadoc` (EPO Inpadoc API)
- `Epo` (EPO Register API)
- `ITCInvestigation` (USITC EDS API)

Several of these models also have sub-models, as discussed below. This notebook shows some simple examples for doing data lookups with each of the models. The other notebooks in the repository show more complex analysis using the [Pandas](https://pandas.pydata.org/) data science library.

## USApplication (USPTO Patent Examination Data System)

The first model we'll look at is USApplication, an abstraction on top of the [USPTO Patent Examination Data System API](https://ped.uspto.gov/peds/#/apiDocumentation). The first step, which is common to all models, is to import the model object - USApplication. 

In [1]:
from patent_client import USApplication

The model can then be queried to get actual data back to do analysis, review, etc. All queries take one of two forms:

- Model.objects.filter(query) -> retreives many objects
- Model.objects.get(query) -> retreives one object, otherwise throws an error

For example, we can query USApplications by applicant by passing the query:
```python
    USApplication.objects.filter(first_named_applicant='Tesla Motors')
```

This query will retrieve all applications that name Tesla Motors as an applicant. By default, if you pass in only an argument without a keyword, like this:

```python
    USApplication.objects.get("10123456")
```
the model assumes that you're referring to the application number, and fetches the corresponding app. A complete list of available query fields is available at Model.objects.query_fields. For example:

```python
    USApplication.objects.query_fields
```

*NOTE:* If you ever need help with something, try calling the Python builtin "help" function. All models have (or should have) super-handy help text about queries, available information, related objects, etc. E.g.:

```python
    help(USApplication)
```  

To protect the innocent, we'll use Tesla's patent portfolio as an example - a company that has publicly agreed not to assert its patents against others. To do a lookup by applicant, we query USApplication object like this:

In [2]:
tesla_apps = USApplication.objects.filter(first_named_applicant='Tesla Motors')
print('We found apps with these applicant names:')
print(list(set(a.applicants[0]['name_line_one'] for a in tesla_apps)))

We found apps with these applicant names:
['TESLA MOTORS. INC.', 'Inc.', 'TESLA MOTORS, INC', 'TESLA MOTORS, INC.', 'Tesla Motors, Inc.']


Let's grab one case, and explore the data therein. Note that all Model objects implement an .attrs method that provides a list of their data attributes:

In [3]:
tesla_app = tesla_apps[15]
print('The following data elements are available:')
print(tesla_app.attrs)

The following data elements are available:
['appl_id', 'applicants', 'app_filing_date', 'app_exam_name', 'inventors', 'app_early_pub_number', 'app_early_pub_date', 'app_location', 'app_grp_art_number', 'patent_number', 'patent_issue_date', 'app_status', 'app_status_date', 'patent_title', 'app_attr_dock_number', 'first_inventor_file', 'app_type', 'app_cust_number', 'app_cls_sub_cls', 'corr_addr_cust_no', 'app_entity_status', 'app_confr_number', 'transaction_history', 'children', 'parents', 'foreign_priority_applications', 'pta_pte_history', 'pta_pte_summary', 'correspondent', 'attorneys']


As you can see, there is a wealth of data about this particular case. Most of the names are relatively self-explanatory. We can look at some basic data by grabbing all the elements that start with "app". First, we'll use the .as_dict() method to convert the application to a standard Python dictionary. Then we'll only print items if they contain "_app":

In [12]:
for k, v in tesla_app.as_dict().items():
    if 'app_' in k or k == 'appl_id':
        print(f'{k}: {v}')
        

appl_id: 13629597
app_filing_date: 2012-09-27
app_exam_name: FOSTER, GERRAD A
app_early_pub_number: US20140088826A1
app_early_pub_date: 2014-03-27
app_location: ELECTRONIC
app_grp_art_number: 3669
app_status: Patented Case
app_status_date: 2014-12-23
app_attr_dock_number: 2011US0045/P0574-2NUS
app_type: Utility
app_cust_number: 113241
app_cls_sub_cls: 701/036000
app_entity_status: UNDISCOUNTED
app_confr_number: 8642


There are also some composite data elements. These elements consist of several related Model objects that model some part of the Application data. These data elements are:
    
- `USApplication.transaction_history` (a list of Transaction objects)
- `USApplication.children` and `USApplication.parents` (both are lists of Relationship objects)
- `USApplication.foreign_priority` (a list of ForeignPriority objects)
- `USApplication.pta_pte_history` (a list of PtaPteHistory objects)
- `USApplication.pta_pte_summary` (a PtaPteSummary object)
- `USApplication.correspondent` (a Correspondent object)
- `USApplication.attorneys` (a list of Attorney objects)
    
To use as an example, let's take a look at the transaction data, and fetch the first 15 records. 

In [5]:
tesla_app.transaction_history[:15]

[<Transaction(date=2012-09-27, description=Initial Exam Team nn)>,
 <Transaction(date=2012-09-28, description=IFW Scan & PACR Auto Security Review)>,
 <Transaction(date=2012-10-01, description=Cleared by OIPE CSR)>,
 <Transaction(date=2012-10-18, description=Filing Receipt)>,
 <Transaction(date=2012-10-18, description=Notice Mailed--Application Incomplete--Filing Date Assigned)>,
 <Transaction(date=2012-10-31, description=Applicants have given acceptable permission for participating foreign)>,
 <Transaction(date=2012-10-31, description=Additional Application Filing Fees)>,
 <Transaction(date=2012-10-31, description=A statement by one or more inventors satisfying the requirement under 35 USC 115, Oath of the Applic)>,
 <Transaction(date=2013-06-25, description=Correspondence Address Change)>,
 <Transaction(date=2013-07-02, description=Application Is Now Complete)>,
 <Transaction(date=2013-07-02, description=Change in Power of Attorney (May Include Associate POA))>,
 <Transaction(date=20

This shows the earliest days of the case, illustrating the USPTO workflow of processing the application from the initial filing date (2012-09-27) to when the case was placed on the examiner's docket (2013-09-23). You might notice that the above data is relatively messy. We can make the data look a bit cleaner by converting the objects to a Pandas dataframe. Every model object in patent_client implements an .as_dict() method that automatically converts the object to an ordinary python dictionary. We can take advantage of that, along with a generator expression, to create a dataframe. This works because each Transaction is itself a Model object:

In [6]:
import pandas as pd
pd.set_option('display.max_colwidth', -1) # makes sure the table runs all the way to the edge

pd.DataFrame.from_records(t.as_dict() for t in tesla_app.transaction_history[:15])

,code,date,description
0,IEXX,2012-09-27,Initial Exam Team nn
1,SCAN,2012-09-28,IFW Scan & PACR Auto Security Review
2,L194,2012-10-01,Cleared by OIPE CSR
3,FLRCPT.O,2012-10-18,Filing Receipt
4,INCD,2012-10-18,Notice Mailed--Application Incomplete--Filing Date Assigned
5,APPERMS,2012-10-31,Applicants have given acceptable permission for participating foreign
6,ADDFLFEE,2012-10-31,Additional Application Filing Fees
7,OATHDECL,2012-10-31,"A statement by one or more inventors satisfying the requirement under 35 USC 115, Oath of the Applic"
8,C.AD,2013-06-25,Correspondence Address Change
9,COMP,2013-07-02,Application Is Now Complete


## PtabTrial (USPTO PTAB Api)

Now that you've seen a little bit of the USApplication API, let's take a look at the PTAB Trials API. We'll start with the usual import:

In [7]:
from patent_client import PtabTrial

For randomness' sake, we'll pick the first IPR of 2017 to play around with, and look it up by its trial number:

In [8]:
trial = PtabTrial.objects.get('IPR2017-00001')
print(f'PTAB Trial No. {trial.trial_number} was filed {trial.filing_date} to challenge U.S. Patent {trial.patent_number}')
print(f'The case is styled {trial.petitioner_party_name} v. {trial.patent_owner_name}')

PTAB Trial No. IPR2017-00001 was filed 2016-10-01 to challenge U.S. Patent 7468661
The case is styled Emerson Electric Co. v. Petite et al


The complete list of available data is again available at .attrs:

In [9]:
print(trial.attrs)

['trial_number', 'application_number', 'patent_number', 'petitioner_party_name', 'patent_owner_name', 'inventor_name', 'prosecution_status', 'filing_date', 'accorded_filing_date', 'institution_decision_date', 'last_modified_datetime', 'documents']


Where possible, there are also related objects that can be fetch from a Model object. For example, the challenged patent has a related USApplication. A complete list of all related objects and other data can be found by calling the help function on the object:

In [10]:
help(trial)

Help on PtabTrial in module patent_client.uspto_ptab object:

class PtabTrial(patent_client.util.Model)
 |  Ptab Trial
 |  ==========
 |  This object wraps the PTAB's public API (https://ptabdata.uspto.gov)
 |  
 |  ---------------------
 |  To Fetch a PTAB Trial
 |  ---------------------
 |  
 |  The main way to create a PtabTrial is by querying the PtabTrial manager at PtabTrial.objects
 |  
 |      PtabTrial.objects.filter(query) -> obtains multiple matching applications
 |      PtabTrial.objects.get(query) -> obtains a single matching application, errors if more than one is retreived
 |  
 |  The query can either be a single number, which is treated as a trial number, or a keyword argument:
 |  
 |      PtabTrial.objects.get('IPR2016-00831') -> Retreives a single trial
 |      PtabTrial.objects.filter(patent_number='6103599') -> retreives all PTAB trials involving US Patent Number 6103599
 |  
 |  A complete list of query fields is available at PtabTrial.objects.query_fields
 |  
 

From the data above, we see that there is a related data object called "us_application". That is, we can fetch the application object that corresponds to the PtabTrial simply by retreiving the attribute .us_application, like so:

In [11]:
trial_app = trial.us_application
print(trial_app)
print({k:v for (k, v) in trial_app.as_dict().items() if 'app_' in k})

<USApplication(appl_id=11395685)>
{'app_filing_date': datetime.date(2006, 3, 31), 'app_exam_name': 'PHAM, TOAN NGOC', 'app_early_pub_number': 'US20060181406A1', 'app_early_pub_date': datetime.date(2006, 8, 17), 'app_location': 'ELECTRONIC', 'app_grp_art_number': '2612', 'app_status': 'Application Involved in Court Proceedings', 'app_status_date': datetime.date(2018, 5, 16), 'app_attr_dock_number': 'STAT3', 'app_type': 'Utility', 'app_cust_number': '89818', 'app_cls_sub_cls': '340/531000', 'app_entity_status': 'UNDISCOUNTED', 'app_confr_number': '1922'}
